In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Define the true function
def true_function(x):
    """y = sin(π*x/2) + x/2"""
    return np.sin((np.pi/2) * x) + x/2


def linear_interpolate_sid(x_data, y_data, x_query):
    """
    Linear interpolation using SID (Version 3) - Pure Python formula-based approach.
    
    Formula:
    y = y_i + (y_{i+1} - y_i) * (x - x_i) / (x_{i+1} - x_i)
    """
    
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    x_query = np.asarray(x_query)
    
    # Ensure data is sorted
    if not np.all(x_data[:-1] <= x_data[1:]):
        sort_indices = np.argsort(x_data)
        x_data = x_data[sort_indices]
        y_data = y_data[sort_indices]
    
    # Initialize result array
    y_interp = np.zeros_like(x_query, dtype=float)
    
    # Process each query point
    for idx, x in enumerate(x_query):
        
        # Extrapolation below data range
        if x <= x_data[0]:
            y_interp[idx] = y_data[0]
            continue
        
        # Extrapolation above data range
        if x >= x_data[-1]:
            y_interp[idx] = y_data[-1]
            continue
        
        # Interpolation within data range
        for i in range(len(x_data) - 1):
            if x_data[i] <= x <= x_data[i + 1]:
                
                x0 = x_data[i]
                x1 = x_data[i + 1]
                y0 = y_data[i]
                y1 = y_data[i + 1]
                
                # Linear interpolation formula
                y_interp[idx] = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
                break
    
    return y_interp


# ============================================================================
# READ DATA FROM FILE
# ============================================================================

file_path = r"D:\\Physics\\Computational Physics and Astrophysics\\Interpolation\\function_data.txt"

x_data = []
y_data = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        parts = line.split()
        if len(parts) >= 2:
            x_data.append(float(parts[0]))
            y_data.append(float(parts[1]))

x_data = np.array(x_data)
y_data = np.array(y_data)

print("="*80)
print("DATA LOADED")
print("="*80)
print(f"Original data points: {len(x_data)}")
print(f"x in [{x_data.min():.1f}, {x_data.max():.1f}]")


DATA LOADED
Original data points: 11
x in [0.0, 10.0]
